In [1126]:
import pandas as pd
pd.__version__

'0.23.0'

# Introduction

## Description

The LoadedTable contains statistics gathered from running the script that generates a matrix (Width x Height) and then 
runs and execution time measured with the [timer:tc](http://erlang.org/doc/man/timer.html#tc-1) following tests for each representation of matrix and sizes:

The tested operations are:

 - one_rows_sums - sum of values from each row in the matrix once per run
 
 - one_cols_sums - sum of values from each row in the matrix once per run
 
 - get_value - gets a value in the middle of the matrix each run separetly
 
 - set_value - sets a value in the middle of the matrix each run separetly

The tested matrix representations are:

In [1127]:
from urllib.request import urlopen
import re
import json

response = urlopen(
    "https://github.com/flmath/matrix_implementations_in_erlang/raw/master/examples/results")
data=[]
for line in response:
    decoded = line.decode('utf-8')
    if "{"==decoded[0]:
        decoded=re.sub('{','{\"',decoded)
        decoded=re.sub(':','\":',decoded)
        decoded=re.sub(', ',', \"',decoded)
        data.append(json.loads(decoded))
        
LoadedTable = pd.DataFrame.from_dict(data, orient='columns')
LoadedTable.head()

,ExecutionTime,Height,Operation,Representation,Runs,Width
0,1078,100,get_value,matrix_as_bit_map,100,100
1,348,100,get_value,matrix_as_ext_bit_map,100,100
2,21,100,get_value,matrix_as_digraph,100,100
3,19,100,get_value,matrix_as_ets_bin,100,100
4,13,100,get_value,matrix_as_map,100,100


Lets get rid of the Width and Runs columns since those are redundant.

In [1128]:
LoadedTable = LoadedTable.loc[:,['ExecutionTime','Height','Operation','Representation']]

Lets list all matrices representations we have tested.

In [1129]:
set(LoadedTable['Representation'])

{'matrix_as_array',
 'matrix_as_array_of_arrays',
 'matrix_as_big_tuple',
 'matrix_as_bit_map',
 'matrix_as_dict',
 'matrix_as_digraph',
 'matrix_as_ets',
 'matrix_as_ets_bin',
 'matrix_as_ets_list',
 'matrix_as_ext_bit_map',
 'matrix_as_gb_tree',
 'matrix_as_list_map',
 'matrix_as_list_of_lists',
 'matrix_as_map',
 'matrix_as_sofs',
 'matrix_as_tuple_of_tuples'}

The matrices are represented in different ways, a matrix can be implemented:

1. As a 'continous memory segment' with values representing Width, Height stored separetly. Element position in the segment is dynamically calculated based on Width, Height. This way of representation is used in: 

 * <span style="color: green">matrix_as_big_tuple</span> - where the matrix values are stored as one tuple of the size Width * Height

 * <span style="color: green">matrix_as_array</span> - where the matrix values are stored as one [array](http://erlang.org/doc/man/array.html) of size Width * Height (note: arrays indexing starts from 0)

2. As a 'dictionary' where everything is stored in form of key-value pairs. The key is created from coordinates pointing to the value. This way of representation is used in: 

 * <span style="color: green">matrix_as_map</span>, <span style="color: green">matrix_as_bit_map</span>, <span style="color: green">matrix_as_ext_bit_map</span> - where the matrix is stored as [map](http://erlang.org/doc/man/maps.html) and keys have form accordingly tuples, binary and again binary. The difference betrween the matrix_as_ext_bit_map and matrix_as_bit_map is that in the former representation the Height and the Width of the matrix are stored in the map and in the latter those values are stored separetly.
 * <span style="color: green">matrix_as_dict</span>, <span style="color: green">matrix_as_gb_trees</span> - since those modules are very similar in usage to the map module they are added to the research
 * <span style="color: green">matrix_as_orddict</span> - orddict has the same interface as dict, but is implemented as one long list, which makes operations extremally time consuming (non of my implementations managed to get resonable execution time, so no results added here).
 * <span style="color: green">matrix_as_ets</span>, <span style="color: green">matrix_as_ets_list</span>, <span style="color: green">matrix_as_ets_bin</span> - where the matrix is stored as [ets table](http://erlang.org/doc/man/ets.html) and key have forms of tuples, lists, and  binaries.

 * <span style="color: green">matrix_as_digraph</span> - uses [digraph](http://erlang.org/doc/man/digraph.html) a directed graph representation where each value is stored in vertex with the coordinate {x,y}. Aside of that each column and row has dedicated vertex with edges from corresponding coordinate vertexes directed at it to speed up rows and columns sums calculations. It is worth mentioning that the digraph is internally implemented with ets table.

3. As a 'compound structure' where we store each row as a element of bigger structure which represent the matrix: 

 * <span style="color: green">matrix_as_list_of_lists</span> - each of rows is a sublist of the list which represents the matrix. Each sublist contains elements representing  values in the row. Since it visually resembles a matrix, it is quite often abused by coders, but is very ineffective since list is implemented as a 'linked list' structure in C language sense. Heavy usage of [lists](http://erlang.org/doc/man/lists.html) module in this implementation. 

 * <span style="color: green">matrix_as_tuple_of_tuples</span> - each of rows is a subtuple of the tuple which represents the matrix. Each subtuple contains elements representing  values in the row. Tuples are implemented as a 'immutable memory block'. The block consist of pointers to numbers or other structures (pointers in C language sense). In this case each pointer references a tuple. Since the tuple is immutable, when we modify it we need to recreate two tuples, the main tuple (matrix) and the tuple containing the value (row tuple). But we do not need to recreate other tuples (rows).
 * <span style="color: green">matrix_as_array_of_arrays</span> - the same as with the tuples but using the array module from OTP.


4. As a relation in strict mathematical sense:

 * <span style="color: green">matrix_as_sofs</span> - Set of the sets module is a way of representing relations (in strictly mathematical sense). In this case the matrix is represented as a relation (a function) F where domain is constructed from coordinates which is related to the matrix entry value ({x,y} F Value).
 
 To get values in specific row we define equivalence relation aRb iff thera are {x,y1} F a {x,y2} F b. This relation divides antidomain into abstract classes. If we sum elements that belongs to an abstract class defined by element {x,_} we will get sum of the elements from the row containing the element x. We use the same logic to sum abstract classes for columns.

 Unfortunatelly the sofs module is implemented with not necessary most efficient usage of the lists. In particular list:fold function is in heavy use in that module.  


Lets find the best solutions for 1000 x 1000 table for specific operations.

In [1130]:
MinTable = (LoadedTable.groupby(['Operation','Representation'])
            .apply(lambda x: pd.Series({'Max': x['ExecutionTime'].max()})))

MinTable.loc[MinTable.groupby(['Operation'])['Max'].idxmin()]

,,Max
Operation,Representation,
cols_sums,matrix_as_big_tuple,11325482
get_value,matrix_as_big_tuple,7
rows_sums,matrix_as_list_of_lists,1124468
set_value,matrix_as_ets_list,23


What we see here is that the best represtentation for retrieving data is keeping values in one big tuple and dimensions of tuple separetly. It is quite intuitive since all you need to get value is calculate memory address and fetch the value.


Rearrange the table for convinience:

In [1131]:
SortedTable = LoadedTable.set_index(['Operation','Representation','Height'])
SortedTable = SortedTable.unstack(2)
IndexTuples = [('ExecutionTime','100'), ('ExecutionTime','200'), ('ExecutionTime','300'), ('ExecutionTime','400'), 
               ('ExecutionTime','500'), ('ExecutionTime','600'), ('ExecutionTime','700'), 
               ('ExecutionTime','800'), ('ExecutionTime','900'), ('ExecutionTime','1000')]
SortedTable = SortedTable.reindex(columns = pd.MultiIndex.from_tuples(IndexTuples))
SortedTable.head()

ExecutionTime                      \
                                              100       200       300   
Operation Representation                                                
cols_sums matrix_as_array                  149107    698647   1673644   
          matrix_as_array_of_arrays        163495    846333   1965130   
          matrix_as_big_tuple               32050     92253    319090   
          matrix_as_bit_map               3810040  28384618  53035475   
          matrix_as_dict                   236984   1311307   4352125   

                                                                      \
                                           400        500        600   
Operation Representation                                               
cols_sums matrix_as_array              3328258    5315118    7969973   
          matrix_as_array_of_arrays    3481540    5531289    8061123   
          matrix_as_big_tuple           366064     625388     863448   
          matrix_as_bit_map          106308550  365437658  226659932   
          matrix_as_dict               9008138   15995373   22315833   

                                                                      \
                                           700        800        900   
Operation Representation                                               
cols_sums matrix_as_array             10883773   14856932   19837491   
          matrix_as_array_of_arrays   11006039   15028130   18675651   
          matrix_as_big_tuple          1234684    2388208    3653456   
          matrix_as_bit_map          349774361  475854253  589586379   
          matrix_as_dict              31890347   42940168   53118754   

                                                 
                                           1000  
Operation Representation                         
cols_sums matrix_as_array              32788776  
          matrix_as_array_of_arrays    22708192  
          matrix_as_big_tuple          11325482  
          matrix_as_bit_map          2484937761  
          matrix_as_dict               67434534

We will be comparing the values of different representations. Since levels in the index is always sorted alphabetically and  sorting just change order of labels, we can use level to create a fixed legend:

In [1132]:
idx = pd.IndexSlice
ColsSums = SortedTable.loc[idx['cols_sums',:],idx[:]]
[(i, ColsSums.index.levels[1][i]) for i in range(0,ColsSums.index.levels[1].size)]

[(0, 'matrix_as_array'),
 (1, 'matrix_as_array_of_arrays'),
 (2, 'matrix_as_big_tuple'),
 (3, 'matrix_as_bit_map'),
 (4, 'matrix_as_dict'),
 (5, 'matrix_as_digraph'),
 (6, 'matrix_as_ets'),
 (7, 'matrix_as_ets_bin'),
 (8, 'matrix_as_ets_list'),
 (9, 'matrix_as_ext_bit_map'),
 (10, 'matrix_as_gb_tree'),
 (11, 'matrix_as_list_map'),
 (12, 'matrix_as_list_of_lists'),
 (13, 'matrix_as_map'),
 (14, 'matrix_as_sofs'),
 (15, 'matrix_as_tuple_of_tuples')]

Lets compare the results of the operations on the borderline small and big matrices:

In [1133]:
ColsSums = SortedTable.loc[idx['cols_sums',:],idx[:]]
(ColsSums.sort_values(by=('ExecutionTime', '1000')).index.labels[1],
ColsSums.sort_values(by=('ExecutionTime', '100')).index.labels[1])

(FrozenNDArray([2, 15, 14, 1, 12, 0, 10, 4, 11, 13, 5, 9, 3, 7, 8, 6], dtype='int8'),
 FrozenNDArray([2, 14, 15, 12, 0, 1, 4, 11, 13, 10, 5, 8, 6, 3, 9, 7], dtype='int8'))

In [1134]:
RowsSums = SortedTable.loc[idx['rows_sums',:],idx[:]]
(RowsSums.sort_values(by=('ExecutionTime', '1000')).index.labels[1],
RowsSums.sort_values(by=('ExecutionTime', '100')).index.labels[1])

(FrozenNDArray([12, 15, 2, 1, 0, 14, 4, 11, 13, 10, 5, 9, 3, 7, 8, 6], dtype='int8'),
 FrozenNDArray([12, 15, 2, 1, 0, 14, 4, 11, 13, 10, 5, 8, 9, 3, 6, 7], dtype='int8'))

In [1135]:
SetValue = SortedTable.loc[idx['set_value',:],idx[:]]
(SetValue.sort_values(by=('ExecutionTime', '1000')).index.labels[1],
SetValue.sort_values(by=('ExecutionTime', '100')).index.labels[1])

(FrozenNDArray([6, 8, 5, 7, 11, 0, 13, 1, 15, 10, 9, 3, 12, 4, 2, 14], dtype='int8'),
 FrozenNDArray([8, 6, 7, 13, 5, 11, 15, 0, 1, 4, 10, 12, 3, 9, 2, 14], dtype='int8'))

In [1136]:
GetValue = SortedTable.loc[idx['get_value',:],idx[:]]
(GetValue.sort_values(by=('ExecutionTime', '1000')).index.labels[1],
GetValue.sort_values(by=('ExecutionTime', '100')).index.labels[1])

(FrozenNDArray([15, 2, 6, 8, 7, 5, 0, 13, 11, 1, 9, 4, 10, 12, 3, 14], dtype='int8'),
 FrozenNDArray([15, 2, 13, 8, 6, 7, 5, 0, 12, 9, 11, 10, 1, 3, 4, 14], dtype='int8'))

Let notify here two observations:

 - We can see that some solutions resource consumtion grows faster than others, even among the best solutions. Like (6, 'matrix_as_ets'), (8, 'matrix_as_ets_list') for set_value.

 - We can see that for some some implementations are leaders for specific operation no matter of the input size, but also there are [dominated](https://en.wikipedia.org/wiki/Multi-objective_optimization) solutions like  (9, 'matrix_as_ext_bit_map').

To address the first problem we should consider solutions that showing resonable growth (less than exponential if possible) of the resource consumption.

From my [projections](https://nbviewer.jupyter.org/urls/github.com/flmath/matrix_implementations_in_erlang/raw/master/jupyter/growth_projections.ipynb) we should consider:
- For rows and columns sums: matrix_as_ets_list and matrix_as_ets representaions.
- For the get_value function the matrix_as_tuple_of_tuples is clear winner, but also the matrix_as_list_of_lists.
- For setting values the results are not so clear, but the matrix_as_array is recomendated. Also matrix_as_dict and matrix_as_sofs should be considered.

Please refer to my other [repo](https://github.com/flmath/empirical-growth-testing) for [methodology](https://nbviewer.jupyter.org/urls/github.com/flmath/empirical-growth-testing/raw/master/Empirical_growth_testing.ipynb) used in the [analysis](https://nbviewer.jupyter.org/urls/github.com/flmath/matrix_implementations_in_erlang/raw/master/jupyter/growth_projections.ipynb).